<a href="https://colab.research.google.com/github/Adenugaoh/The-Healthcare-Service-Finder/blob/main/The_Heallthcare_Service_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Healthcare Service Finder


In [ ]:
#Considering to provide a tool that be useful for finding nearby healthcare facilities
# The Healthcare Service Finder Bot
# Importing necessary libraries
import re
import random
import nltk
import requests
import json
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
except:
    pass

# Healthcare service types listed
healthcare_services = {
    "general": ["general practitioner", "family doctor", "primary care"],
    "emergency": ["emergency room", "urgent care", "emergency department"],
    "specialist": ["cardiologist", "dermatologist", "orthopedic", "neurologist"],
    "dental": ["dentist", "dental clinic", "orthodontist"],
    "mental_health": ["psychiatrist", "psychologist", "counselor", "therapist"],
    "pharmacy": ["pharmacy", "chemist", "drug store"],
    "hospital": ["hospital", "medical center"],
    "pediatric": ["pediatrician", "children's doctor", "pediatric"],
    "women_health": ["gynecologist", "obstetrician", "women's health"],
    "eye_care": ["ophthalmologist", "optometrist", "eye doctor"]
}

# Predefined intents and responses
intents = {
    "greeting": ["Hello! I'm here to help you find healthcare services near you.",
                "Hi there! I can help you locate healthcare providers in your area.",
                "Hey! How can I assist you with finding healthcare services?"],
    "goodbye": ["Take care of your health! Goodbye!", "Stay healthy! See you soon!", "Bye! Remember to prioritize your health!"],
    "thanks": ["You're welcome! Stay healthy!", "No problem! Hope you get the care you need!", "Glad to help with your healthcare needs!"],
    "help": ["I can help you find healthcare providers near you. Just tell me what type of care you need and your location!"],
    "ask_care_type": ["What type of healthcare service are you looking for? (e.g., general doctor, emergency care, dentist, specialist)"],
    "ask_location": ["Please provide your location (city, state or post code):"],
    "fallback": ["I'm sorry, I didn't understand. I can help you find healthcare providers. What type of care do you need?"]
}

# Keyword patterns for detecting intent
patterns = {
    "greeting": r"hello|hi|hey|start",
    "goodbye": r"bye|see you|goodbye|exit|quit",
    "thanks": r"thanks|thank you",
    "help": r"help|assist|support",
    "healthcare_request": r"doctor|hospital|clinic|medical|health|care|physician|emergency|dentist|pharmacy"
}

class TheHealthcareServiceFinder:
    def __init__(self):
        self.conversation_state = "initial"
        self.user_location = None
        self.care_type = None
        self.geolocator = Nominatim(user_agent="healthcare_chatbot")

    def get_intent(self, user_input):
        for intent, pattern in patterns.items():
            if re.search(pattern, user_input.lower()):
                return intent
        return "fallback"

    def identify_care_type(self, user_input):
        user_input_lower = user_input.lower()
        for service_type, keywords in healthcare_services.items():
            for keyword in keywords:
                if keyword in user_input_lower:
                    return service_type
        return None

    def get_user_location(self, location_input):
        try:
            location = self.geolocator.geocode(location_input)
            if location:
                return (location.latitude, location.longitude), location.address
            else:
                return None, None
        except:
            return None, None

    def search_healthcare_providers(self, care_type, location_coords):
        """
        Search for healthcare providers using OpenStreetMap Overpass API (free)
        """
        lat, lon = location_coords

        # Define search terms based on care type
        search_terms = {
            "general": "amenity=doctors",
            "emergency": "amenity=hospital",
            "specialist": "amenity=doctors",
            "dental": "amenity=dentist",
            "mental_health": "amenity=doctors",
            "pharmacy": "amenity=pharmacy",
            "hospital": "amenity=hospital",
            "pediatric": "amenity=doctors",
            "women_health": "amenity=doctors",
            "eye_care": "amenity=doctors"
        }

        search_term = search_terms.get(care_type, "amenity=doctors")

        # Overpass API query (searches within ~10 miles / 16 km)
        overpass_query = f"""
        [out:json][timeout:25];
        (
          node[{search_term}](around:16000,{lat},{lon});
          way[{search_term}](around:16000,{lat},{lon});
          relation[{search_term}](around:16000,{lat},{lon});
        );
        out center meta;
        """

        try:
            response = requests.post(
                "https://overpass-api.de/api/interpreter",
                data=overpass_query,
                timeout=30
            )

            if response.status_code == 200:
                data = response.json()
                providers = []

                for element in data.get('elements', []):
                    if 'tags' in element:
                        tags = element['tags']
                        name = tags.get('name', 'Healthcare Provider')

                        # Get coordinates
                        if element['type'] == 'node':
                            provider_lat = element['lat']
                            provider_lon = element['lon']
                        elif 'center' in element:
                            provider_lat = element['center']['lat']
                            provider_lon = element['center']['lon']
                        else:
                            continue

                        # Calculate distance
                        distance = geodesic(location_coords, (provider_lat, provider_lon)).miles

                        if distance <= 10:  # Within 10 miles
                            provider_info = {
                                'name': name,
                                'distance': round(distance, 1),
                                'lat': provider_lat,
                                'lon': provider_lon,
                                'address': tags.get('addr:full', 'Address not available'),
                                'phone': tags.get('phone', 'Phone not available'),
                                'website': tags.get('website', 'Website not available')
                            }
                            providers.append(provider_info)

                # Sort by distance
                providers.sort(key=lambda x: x['distance'])
                return providers[:10]  # Return top 10 closest

        except Exception as e:
            print(f"Error searching providers: {e}")
            return []

        return []

    def format_provider_response(self, providers, care_type):
        if not providers:
            return f"I'm sorry, I couldn't find any {care_type} providers within 10 miles of your location. You might want to try:\n1. Expanding your search radius\n2. Checking with your insurance provider\n3. Calling 211 for local healthcare resources"

        response = f"Here are {len(providers)} {care_type} healthcare providers within 10 miles of your location:\n\n"

        for i, provider in enumerate(providers, 1):
            response += f"{i}. {provider['name']}\n"
            response += f"   Distance: {provider['distance']} miles\n"
            response += f"   Address: {provider['address']}\n"
            response += f"   Phone: {provider['phone']}\n"
            if provider['website'] != 'Website not available':
                response += f"   Website: {provider['website']}\n"
            response += f"   Maps: https://www.google.com/maps?q={provider['lat']},{provider['lon']}\n\n"

        response += "💡 Tips:\n"
        response += "• Call ahead to verify hours and availability\n"
        response += "• Check if they accept your insurance\n"
        response += "• For emergencies, call 911 or go to the nearest emergency room\n"
        response += "• For non-emergency medical advice, consider telehealth options"

        return response

    def respond(self, user_input):
        intent = self.get_intent(user_input)

        if self.conversation_state == "initial":
            if intent == "greeting":
                self.conversation_state = "asking_care_type"
                return random.choice(intents["greeting"]) + "\n" + random.choice(intents["ask_care_type"])
            elif intent == "healthcare_request":
                self.conversation_state = "asking_care_type"
                return random.choice(intents["ask_care_type"])
            elif intent == "help":
                return random.choice(intents["help"])
            elif intent == "goodbye":
                return random.choice(intents["goodbye"])
            elif intent == "thanks":
                return random.choice(intents["thanks"])
            else:
                return random.choice(intents["fallback"])

        elif self.conversation_state == "asking_care_type":
            care_type = self.identify_care_type(user_input)
            if care_type:
                self.care_type = care_type
                self.conversation_state = "asking_location"
                return f"Great! You're looking for {care_type} care. " + random.choice(intents["ask_location"])
            else:
                service_list = ", ".join([f"{k} ({', '.join(v[:2])})" for k, v in list(healthcare_services.items())[:5]])
                return f"I didn't recognize that healthcare service. Here are some options I can help with:\n{service_list}\n\nWhat type of care do you need?"

        elif self.conversation_state == "asking_location":
            location_coords, location_address = self.get_user_location(user_input)
            if location_coords:
                self.user_location = location_coords
                self.conversation_state = "searching"

                print("Searching for healthcare providers near you...")
                providers = self.search_healthcare_providers(self.care_type, location_coords)

                response = f"Found your location: {location_address}\n\n"
                response += self.format_provider_response(providers, self.care_type)
                response += "\n\nWould you like to search for a different type of care or location? (Type 'new search' to start over)"

                self.conversation_state = "completed"
                return response
            else:
                return "I couldn't find that location. Please try again with a city, state, or post code (e.g., 'New York, NY' or '90210'):"

        elif self.conversation_state == "completed":
            if "new search" in user_input.lower() or "another" in user_input.lower():
                self.conversation_state = "asking_care_type"
                self.care_type = None
                self.user_location = None
                return "Let's start a new search! " + random.choice(intents["ask_care_type"])
            elif intent == "goodbye":
                return random.choice(intents["goodbye"])
            elif intent == "thanks":
                return random.choice(intents["thanks"])
            else:
                return "I can help you with another healthcare search. Type 'new search' to start over, or 'bye' to exit."

        return random.choice(intents["fallback"])

# Main chatbot execution
def run_chatbot():
    chatbot = TheHealthcareServiceFinder()
    print("The Healthcare Service Finder: Hello! I can help you find healthcare providers near you.")
    print("Type 'bye' to end the chat.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['bye', 'exit', 'quit', 'done']:
            print("Chatbot:", random.choice(intents["goodbye"]))
            break

        response = chatbot.respond(user_input)
        print("Chatbot:", response)
        print()

if __name__ == "__main__":
    # Install required packages if not already installed
    try:
        import geopy
    except ImportError:
        print("Installing required packages...")
        import subprocess
        subprocess.check_call(["pip", "install", "geopy", "requests"])
        import geopy

    run_chatbot()